#  Testing through documentation

26.3 doctest — Test interactive Python examples   
https://docs.python.org/3/library/doctest.html

doctest – Testing through documentation   
https://pymotw.com/2/doctest/index.html

<b>doctest</b> lets you <b>test</b> your code by running <b>examples embedded in the documentation</b> and verifying that they produce the expected results. 

It works by parsing the help text to find examples, running them, then comparing the output text against the expected value. 

Many developers find doctest <b>easier</b> than unittest because in its simplest form, there is no API to learn before using it.

However, as the examples become more complex <b>the lack of fixture management</b> can make writing doctest tests more <b>cumbersome</b> than using unittest.

### 1 Getting Started

<b>doctest</b> looks for lines <b>beginning</b> with 

the interpreter prompt, <b>>>></b>, to find the beginning of a test case. 

The case is <b>ended</b> 
      
       by <b>a blank line</b>, 
      
      or by the <b>next interpreter prompt</b>.

Here, <b>my_function()</b> has two examples given in the module: doctest_simple.py

In [ ]:
def my_function(a, b):
    """
    >>> my_function(2, 3)
    6
    >>> my_function('a', 3)
    'aaa'
    """
    return a * b

To run the tests, use <b>doctest as the main program</b> via the <b>-m</b> option to the interpreter:

In [ ]:
!python -m doctest ..\code\doctest\doctest_simple.py

Usually no output is produced while the tests are running,

so the example below includes the <b>-v</b> option to make the output more verbose.

In [ ]:
!python -m doctest -v ..\code\doctest\doctest_simple.py

Examples cannot usually stand on their own as explanations of a function, so doctest also lets you keep the surrounding text you would normally include in the documentation. 

Intervening text is ignored, and can have any format as long as it does not look like a test case.



In [ ]:
def my_function(a, b):
    """Returns a * b.

    Works with numbers:
    
    >>> my_function(2, 3)
    6

    and strings:
    
    >>> my_function('a', 3)
    'aaa'
    """
    return a * b

The surrounding text in the updated docstring makes it more <b>useful to a human reader</b>, and is  <b>ignored by doctest</b>, and the results are the same.

In [ ]:
!python -m doctest -v ..\code\doctest\doctest_simple_with_docs.py

## 2 Handling Unpredictable Output

There are other cases where the <b>exact output may not be predictable</b>, but should still be testable.

* Local date and time values and object ids <b>change</b> on every test run. 

* The default precision used in the representation of floating point values depend on compiler options.

* Object string representations may not be deterministic. 


In [ ]:
class MyClass(object):
    pass

def unpredictable(obj):
    """Returns a new list containing obj.

    >>> unpredictable(MyClass())
    [<doctest_unpredictable.MyClass object at 0x10055a2d0>]
    """
    return [obj]

These `id` values change each time a program runs, because it is loaded into a different part of memory.

In [ ]:
!python -m doctest -v ..\code\doctest\doctest_unpredictable.py

When the tests include values that are likely to <b>change in unpredictable ways</b>, and where the actual value is not important to the test results,

you can use the <b>ELLIPSIS</b> option to tell `doctest` to ignore portions of the verification value.

The comment after the call to `unpredictable()` (#doctest: +ELLIPSIS) tells `doctest` to turn on the ELLIPSIS option for that test. 
The `...` replaces `the memory address` in the object id, so that portion of the expected value is ignored and the actual output matches and the test passes.

In [ ]:
!python -m doctest -v ..\code\doctest\doctest_ellipsis.py

#### There are cases where you cannot ignore the unpredictable value, because that would obviate the test.

For example, simple tests quickly become more complex when dealing with data types whose string representations are inconsistent. 

The string form of a dictionary, for example, may <b>change based on the order the keys are added<b>.

In [ ]:
keys = [ 'a', 'aa', 'aaa' ]

d1 = dict( (k,len(k)) for k in keys )

d2 = dict( (k,len(k)) for k in reversed(keys) )

print
print('d1:', d1)
print('d2:', d2)
print('d1 == d2:', d1 == d2)

s1 = set(keys)
s2 = set(reversed(keys))

print
print('\ns1:', s1)
print('s2:', s2)
print('s1 == s2:', s1 == s2)


Because of <b>cache collision</b>, the internal <b>key list order is different</b> for the two dictionaries, evencthough they contain the same values and are considered to be equal.

<b>Sets</b> use the same hashing algorithm, and exhibit the same behavior.

#### The best way to deal with these potential discrepancies is

to create tests that produce values that are not likely to change. 

In the case of `dictionaries` and `sets`, that might mean looking for <b>specific keys</b> individually, generating a sorted list of the contents of the data structure, or comparing against <b>a literal value</b> for equality instead of depending on <b>the string representation</b>.

In [ ]:
def group_by_length(words):
    """Returns a dictionary grouping words into sets by length.

    >>> grouped = group_by_length([ 'python', 'module', 'of', 'the', 'week' ])
    >>> grouped == { 2:set(['of']),
    ...              3:set(['the']),
    ...              4:set(['week']),
    ...              6:set(['python', 'module']),
    ...              }
    True

    """
    
    d = {}
    for word in words:
        s = d.setdefault(len(word), set())
        s.add(word)
    return d

Notice that the single example is actually interpreted as two separate tests, with 

the first expecting no console output and 

the second expecting the boolean result of the comparison operation.

In [ ]:
!python -m doctest -v ..\code\doctest\doctest_hashed_values_tests.py

## 3 Tracebacks

Tracebacks are a special case of changing data. Since the paths in a traceback depend on the location where a module is installed on the filesystem on a given system, it would be impossible to write portable tests if they were treated the same as other output.

In [ ]:
def this_raises():
    """This function always raises an exception.

    >>> this_raises()
    Traceback (most recent call last):
      File "<stdin>", line 1, in <module>
      File "/no/such/path/doctest_tracebacks.py", line 14, in this_raises
        raise RuntimeError('here is the error')
    RuntimeError: here is the error
    """
    raise RuntimeError('here is the error')

doctest makes a special effort to recognize tracebacks, and ignore the parts that might change from system to system.

In [ ]:
 !python -m doctest -v ..\code\doctest\doctest_tracebacks.py